In [7]:
import tensorflow as tf;
from keras.preprocessing.image import ImageDataGenerator;

tf.__version__

'2.15.0'

Part1 - Data Pre-Processing

In [8]:
#Preprocess training set

#init inmage augmentation engine.
train_datagen = ImageDataGenerator(
    rescale=1./255,#feature scaling
    shear_range=0.2,#augumentation
    zoom_range=0.2,#augumentation
    horizontal_flip=True#augumentation
)

training_set = train_datagen.flow_from_directory(
    'training_set', #dir folder
    target_size=(64,64),#standard 150 reduce size for speed.
    batch_size=32,
    class_mode='binary'
)

Found 8000 images belonging to 2 classes.


In [9]:
#test set is ONLY rescaled to above value.
test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_directory(
    'test_set', #dir folder
    target_size=(64,64),#same format as above
    batch_size=32,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.


Build the CNN

In [10]:

#Init the CNN

cnn = tf.keras.Sequential()

#---Convolution 1---
#filters amount of Kernel klusters, kernel - amount of kernels in the cluster 3x3 below, 
# input shape has to correlate to above size. Last index in input shape arr,  3 for rgb 1 for grey
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu", input_shape=[64, 64, 3]))

#---Pooling----
#pool size similar to filter above but this time on the feature map.
#stride - should correlate to pool size as the feature map is scanned.
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

#---Convolution 2---
#next conv and pooling step (always pool after conv). Remove input shape from second onward
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu"))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

#----Flattening----
#once convolutions are done, the matrix is flattended to be fed into the ANN
cnn.add(tf.keras.layers.Flatten())

# ----Full Connection - add regular ANN-----
#hidden layer
cnn.add(tf.keras.layers.Dense(units=128, activation="relu"))

#output layer
cnn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))




In [11]:
#compile the CNN
#optimizer adam performs stochastic gradient descent when updating the weight via backpropagation
cnn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

#training the CNN
cnn.fit(x=training_set, validation_data=test_set, epochs=25)

Epoch 1/25
250/250 [==============================] - 50s 196ms/step - loss: 0.6523 - accuracy: 0.6049 - val_loss: 0.6764 - val_accuracy: 0.6030
Epoch 2/25
250/250 [==============================] - 46s 183ms/step - loss: 0.5794 - accuracy: 0.6940 - val_loss: 0.5439 - val_accuracy: 0.7200
Epoch 3/25
250/250 [==============================] - 46s 185ms/step - loss: 0.5506 - accuracy: 0.7207 - val_loss: 0.5166 - val_accuracy: 0.7545
Epoch 4/25
250/250 [==============================] - 48s 191ms/step - loss: 0.5178 - accuracy: 0.7401 - val_loss: 0.5019 - val_accuracy: 0.7565
Epoch 5/25
250/250 [==============================] - 48s 192ms/step - loss: 0.4897 - accuracy: 0.7586 - val_loss: 0.4975 - val_accuracy: 0.7605
Epoch 6/25
250/250 [==============================] - 50s 199ms/step - loss: 0.4703 - accuracy: 0.7709 - val_loss: 0.5231 - val_accuracy: 0.7505
Epoch 7/25
250/250 [==============================] - 49s 194ms/step - loss: 0.4573 - accuracy: 0.7820 - val_loss: 0.4688 - val_ac

In [18]:
import numpy as np;
from keras.utils import load_img, img_to_array;

#load the test image
test_image = load_img('single_prediction/cat_or_dog_2.jpg', target_size=(64,64))

#turn loaded image into 2d numpy array
test_image = img_to_array(test_image)

#We created a batch of 32 images above so we have an extra dimension in the array
#so we have to turn the single image above into a batch - by adding an extra dimension
#the batch is the first dimension since it contains the other arrays so axis is 0
test_image = np.expand_dims(test_image, axis=0)

#predict the performance of the trained array - the image has to be normalized to 255
result=cnn.predict(test_image/255.0)

#the output will be binary so how do we know if 0/1 is cat or dog?
#we call the training_set and call the class_indices.

#[0] is the batch then[0] is the element of the prediction
if result[0][0] > 0.5:
    prediction = "dog"
else:
    prediction = "cat"

print(prediction)



1/1 [==============================] - 0s 29ms/step
cat
